In [1]:
import pandas as pd
import os
import glob

In [2]:
def createDirectory(directory): 
    """ 새로운 폴더를 생성하는 함수 """
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print("Error: Failed to create the directory.")

In [3]:
def show(d):
    """ 위원 목록, 중복 인원 수를 보여주는 함수 """
    u = d['발언자'].unique()
    u2 = [w[:3] if w[-2:] == "위원" else w[-3:] for w in u]
    set_u2 = set(u2)
    li_set_u2 = list(set_u2)
    dup_cnt = len(u2) - len(set_u2)
    print("==== 위원 리스트 ====")
    print(sorted(u2))
    print("==== 중복 인원수 ====")
    print(dup_cnt)
    dup_list = [p for p in set_u2 if u2.count(p) > 1]
    print("==== 중복 인원 ====")
    print(sorted(dup_list))
    print()

In [4]:
def freq_plus(p1, p2):
    """ 중복 인원에 대한 여러 파일의 단어 빈도수를 합산하여 반환하는 함수. """
    wf = p1[['단어', '빈도수']]
    name = p1['의원명'].iloc[0]
    zip_dict =  dict([(a,b) for a,b in zip(wf['단어'],wf['빈도수'])])

    for i in range(len(p2)):
        word = p2.iloc[i, 2]
        freq = p2.iloc[i, 3]
        if word in zip_dict:
            zip_dict[word] += freq
        else:
            zip_dict[word] = freq
    
    return make_df_person(zip_dict, name)

In [8]:
def make_df_person(c, p_name):
    """ 정해진 포맷에 맞게 데이터프레임을 생성해주는 함수. (의원 추가) """
    
    d = pd.DataFrame.from_dict(c, orient='index').reset_index()
    
    ''' 발언한 적은 있지만 명사에 대한 결과가 없는 경우 단어가 잡히지 않음.
    이 경우 빈 데이터프레임으로 나와 빈 데이터프레임을 반환하는 것으로 예외처리함. 
    ex) 전체 회의에서 '네, 알겠습니다.' 만 대답한 경우 '''
    try:
        d.columns = ['단어', '빈도수']
    except ValueError:
        # 빈 데이터프레임 반환
        return pd.DataFrame()
    
    count_column = df['회수']
    d['의원명'] = p_name
    d = pd.concat([d, count_column], axis = 1).dropna()
    d = d[['회수', '의원명', '단어', '빈도수']]
    
    return d  

### 2017년 이름 중 '양천)'은 이름 형식을 지키지 않은 예외 케이스. 중복 인원이 아닌 것을 확인함.

In [5]:
location = os.getcwd()

fname_list = os.listdir(os.path.join(location, "회의록_모음"))
file_list = glob.glob(os.path.join(location, "회의록_모음/*"))

# 원하는 폴더 이름 설정.
dir_name = "단어 빈도 파일 중복 인물 통합"
createDirectory(dir_name)

for i in range(len(fname_list)):
    df = pd.read_csv(file_list[i], encoding = 'euc-kr')
    print(fname_list[i][:-4])
    show(df)

서울특별시의회 회의록 제277회[정례회] (2017.11.01 ~ 2017.12.20)
==== 위원 리스트 ====
['강구덕', '강성언', '김광선', '김동욱', '김병혁', '김생환', '김석일', '김선주', '김선희', '김수득', '김시영', '김양주', '김재숙', '김재숙', '김진수', '김형태', '김희선', '문형주', '민정숙', '박기열', '박연선', '박혜자', '백종대', '손영순', '송재형', '송재형', '신재웅', '안은용', '안재홍', '양천)', '양희두', '오경환', '유재준', '윤석명', '윤오영', '이강태', '이경희', '이길환', '이무수', '이민종', '이상수', '이숙자', '이연주', '이용환', '이재관', '이재근', '이정훈', '이종배', '임세훈', '임영식', '장인홍', '장인홍', '전병화', '전종보', '전창신', '정병익', '정병익', '정인순', '정재헌', '정해철', '정혜손', '정호영', '조영상', '조원익', '조희연', '진영학', '최광락', '최영식', '허기회', '홍민표', '황현택']
==== 중복 인원수 ====
4
==== 중복 인원 ====
['김재숙', '송재형', '장인홍', '정병익']

서울특별시의회 회의록 제284회[정례회] (2018.11.01 ~ 2018.12.20)
==== 위원 리스트 ====
[' 전희', '강동호', '강연흥', '권미애', '권성연', '권순선', '김경 ', '김광균', '김광미', '김수규', '김승영', '김시영', '김양주', '김원찬', '김진수', '김현철', '김형태', '민계홍', '민병관', '박건호', '박건호', '박광훈', '박혜자', '백종대', '서경수', '송재범', '신승인', '심금순', '양민규', '양희두', '여명 ', '윤오영', '이무수', '이민종', '이병호', '이연주', '이재근', '이정순', '이주영', '이혜련', '임광빈', '임찬식', '장상기', '장석

### 중복 인물이 아닌 동명이인일 수 있어서 해당 부분은 통합 원하는 파일을 설정하고 진행해야 함.

In [9]:
# 파일이 존재하는 폴더 이름으로 경로설정해주고 불러와야 함.
folder = "위원 별 단어 빈도 분석"
loc = os.path.join(location, folder)
"""
원하는 년도에 따라 fname_list의 원소를 바꾸어 대입하자. -> fname_list는 위원 별 단어 빈도 분석의 폴더명과 같다는 전제하에 설정.
결국 pick 변수에 들어갈 경로는 년도별 위원 별 단어 빈도 분석 폴더 경로여야 한다는 것을 기억해야 함.
""" 
pick = os.path.join(loc, fname_list[0][:-4])

# csv파일 이름은 직접 입력하기.
f1 = pd.read_csv(os.path.join(pick, "장인홍 위원.csv"), encoding = 'euc-kr')
f2 = pd.read_csv(os.path.join(pick, "부위원장 장인홍.csv"), encoding = 'euc-kr')
result = freq_plus(f1, f2)

# csv파일 이름은 직접 입력하기.
result.to_csv(os.path.join(dir_name, "장인홍_통합.csv"), index = False, encoding = 'euc-kr')